In [213]:
import pandas as pd
import re
from datetime import datetime

## Preprocess the data

In [214]:
raw_data_path = './dataset/raw_data.csv'
wind_list_path = './dataset/wind_list.pkl'

### Preprocess the data
###     1. process birthday attribute with dropping the place of the birth in the string
###     2. process record of the performance with replacing '-' into '-1 (h0)'
###     3. extract weight and height attribute from body and drop the rows with no body information
###     4. drop some unuseful columns

raw_data = pd.read_csv(raw_data_path)
raw_data.dropna(subset=['Birth', 'Body', 'Year'], inplace=True)
raw_data['Birthday'] = raw_data['Birth'].apply(lambda x: x.split("in")[0])
raw_data.loc[raw_data['R1'] == '–', 'R1'] = '-1 (h0)'
raw_data.loc[raw_data['R2'] == '–', 'R2'] = '-1 (h0)'
raw_data.loc[raw_data['R3'] == '–', 'R3'] = '-1 (h0)'
drop_index = []
for idx in raw_data.index:
    try:
        raw_data.loc[idx, 'Height'] = int(raw_data['Body'][idx].split()[0])
        raw_data.loc[idx, 'Weight'] = int(raw_data['Body'][idx].split()[3])
    except:
        drop_index.append(idx)
raw_data.drop(index=drop_index, inplace=True)
raw_data.drop(columns=['Gold', 'Silver', 'Bronze', 'Birth', 'Body'], inplace=True)

### Process the information of wind

wind_list = pd.read_pickle(wind_list_path)
wind_dict = {}
for idx in range(len(wind_list)):
    wind_dict[1948 + idx * 4] = wind_list[idx]
print(wind_dict)

### Process the data of performance record
###     1. to make sure the data could be convert to float type
###     2. extract the heat information

raw_data['R1'] = raw_data['R1'].apply(lambda x: re.sub(r'[\[\]w]', '', x))
raw_data['R2'] = raw_data['R2'].apply(lambda x: re.sub(r'[\[\]w]', '', x))
raw_data['R3'] = raw_data['R3'].apply(lambda x: re.sub(r'[\[\]w]', '', x))

raw_data['R1'] = raw_data['R1'].apply(lambda x: re.sub('–', '-1', x))
raw_data['R2'] = raw_data['R2'].apply(lambda x: re.sub('–', '-1', x))
raw_data['R3'] = raw_data['R3'].apply(lambda x: re.sub('–', '-1', x))
raw_data['R4'] = raw_data['R3'].apply(lambda x: re.sub('–', '-1', x))
for i in raw_data.index:
    p1 = raw_data['R1'][i].find('h')
    p2 = raw_data['R2'][i].find('h')
    p3 = raw_data['R3'][i].find('h')
    raw_data.loc[i, 'heat_r1'] = int(raw_data['R1'][i][p1+1:-1])
    raw_data.loc[i, 'heat_r2'] = int(raw_data['R2'][i][p2+1:-1])
    raw_data.loc[i, 'heat_r3'] = int(raw_data['R3'][i][p3+1:-1])
    p1 = raw_data['R1'][i].find('(')
    p2 = raw_data['R2'][i].find('(')
    p3 = raw_data['R3'][i].find('(')
    p4 = raw_data['R4'][i].find('(')
    raw_data.loc[i, 'R1'] = float(raw_data['R1'][i][:p1])
    raw_data.loc[i, 'R2'] = float(raw_data['R2'][i][:p2])
    raw_data.loc[i, 'R3'] = float(raw_data['R3'][i][:p3])
    raw_data.loc[i, 'R4'] = float(raw_data['R4'][i][:p3])


num_heat = [raw_data.groupby('Year')['heat_r1'].max(), # use to match the wind information
            raw_data.groupby('Year')['heat_r2'].max(), 
            raw_data.groupby('Year')['heat_r3'].max()]
print(raw_data)

{1948: [1.1, 0.3, 0.4, 1.3, 0.9, 1.0, 0.9, 0.3, 0.6, 3.3, 0.9, 0.2, 0.0, 1.5, 1.9, 1.5, 1.3, 1.6], 1952: [], 1956: [0.0, -1.2, -2.8, 0.0, 0.1, -0.5, -0.6, 0.0, -0.9, -1.2, -0.7, 0.0, -1.4, 0.0, -1.0, -2.2, -2.3, -1.1, -2.5], 1960: [-0.3, -0.2, -0.3, -0.2, -0.2, -0.5, -0.3, -0.1, -0.2, -0.5, -0.3, -1.3, -2.3, 0.0, 0.0, 0.0], 1964: [0.6, -2.5, -2.8, -0.7, 0.3, 1.6, -1.8, 0.2, 0.2, -0.5, 1.9, 1.7, 1.0, 1.7, 5.3, -1.3, 1.0], 1968: [2.8, 0.8, 0.0, 0.6, 0.7, 3.8, 0.4, 0.0, 0.0, 1.8, 0.5, 4.2, 2.0, 1.6, 0.0, 0.3], 1972: [-0.7, -2.3, 0.5, 2.3, 0.8, 1.2, -1.9, 2.1, 0.6, -0.3, -2.2, -0.3, 1.8, -2.3, 0.0, 3.4, 0.3, 0.0, 0.2, 0.3], 1976: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.1, 0.1, 0.1, 0.1, 1.1, 0.7, 0.0], 1980: [0.0, 0.1, 0.5, -0.2, 0.9, 0.0, -0.1, -0.1, 0.3, 1.4, 0.2, 0.3, 0.3, 0.5, 0.5, 1.2], 1984: [-0.4, 1.8, 1.8, -0.8, 1.4, 1.9, 0.8, -0.8, -1.7, -1.4, 1.4, -0.7, 0.1, 1.4, 0.8, 0.8, 0.7, -1.5, 0.2], 1988: [0.6, 0.9, 0.7, 0.8, 1.1, 1.4, 1.8, 2.0, 1.0, 1.4, 1.0, 1.4, 0.9, 1.2, 1.7, 0

## Process the data

### Extract train data from raw data

In [215]:
train_data = pd.DataFrame(columns=['Name', 'NOC', 'Weight', 'Height', 'Birthday' , 'Year', 'Round', 'Wind', 'Label'])
for _, row in raw_data.iterrows():
    for round in range(1, 5):
        col = f'R{round}'
        if row[col] == -1:
            continue
        try:
            wind_idx = 0 if round == 1 else num_heat[round - 2][row['Year']]
            wind_idx = wind_idx + row[f'heat_r{round}'] - 1 if round != 4 else -1
            wind = wind_dict[row['Year']][int(wind_idx)]
        except:
            break
        new_row = list(row[['Name', 'Nation', 'Weight', 'Height', 'Birthday' , 'Year']])
        new_row += [round, wind, row[col]]
        train_data.loc[len(train_data)] = new_row
train_data.dropna(inplace=True)
train_data = train_data.loc[train_data['Label'] <= 11.5]
print(train_data)

                  Name  NOC  Weight  Height           Birthday  Year  Round  \
0     Harrison Dillard  USA    69.0   178.0       8 July 1923   1948      1   
1     Harrison Dillard  USA    69.0   178.0       8 July 1923   1948      2   
2     Harrison Dillard  USA    69.0   178.0       8 July 1923   1948      3   
3     Harrison Dillard  USA    69.0   178.0       8 July 1923   1948      4   
4         Barney Ewell  USA    71.0   180.0  25 February 1918   1948      1   
...                ...  ...     ...     ...                ...   ...    ...   
2291       Didier Kiki  BEN    84.0   185.0  30 November 1995   2020      1   
2292    Karalo Maibuca  TUV    64.0   176.0      10 June 1999   2020      1   
2293     Chijindu Ujah  GBR    81.0   182.0      5 March 1994   2020      2   
2294     Chijindu Ujah  GBR    81.0   182.0      5 March 1994   2020      3   
2295     Chijindu Ujah  GBR    81.0   182.0      5 March 1994   2020      4   

      Wind  Label  
0      0.9  10.40  
1      0.0 

### Process the train data

In [216]:
import calendar
olympic_list = {
    1948: ['GBR', '19480730'],
    1952: ['FIN', '19520721'],
    1956: ['AUS', '19561123'],
    1960: ['ITA', '19600831'],
    1964: ['JPN', '19641014'],
    1968: ['MEX', '19681013'], 
    1972: ['GER', '19720831'],
    1976: ['CAN', '19760723'],
    1980: ['URS', '19800724'],
    1984: ['USA', '19840803'],
    1988: ['KOR', '19880923'],
    1992: ['ESP', '19920731'],
    1996: ['USA', '19960726'],
    2000: ['AUS', '20000922'],
    2004: ['GRE', '20040821'],
    2008: ['CHN', '20080815'],
    2012: ['GBR', '20120804'],
    2016: ['BRA', '20160813'],
    2020: ['JPN', '20210731'],
}



drop_index = []
for idx, row in train_data.iterrows():
    birthday = row['Birthday']
    try:
        month = birthday.split()[1]
    except:
        drop_index.append(idx)
        continue
    birthday = birthday.replace(month, str(list(calendar.month_name).index(month)))
    while birthday[-1] == ' ':
        birthday = birthday[:-1]
    birthday = datetime.strptime(birthday, "%d %m %Y")
    train_data.loc[idx, "Age"] = (datetime.strptime(olympic_list[row['Year']][1], "%Y%m%d") - birthday).days / 365.25
    train_data.loc[idx, "isHometown"] = int(row['NOC'] == olympic_list[row['Year']][0])
    train_data.loc[idx, "BMI"] = row['Weight'] * 10000 / row['Height'] / row['Height']
train_data.drop(index=drop_index, inplace=True)
train_data.dropna(inplace=True)
nation_cnt = dict(train_data.groupby(by=['NOC']).count()['Label'].sort_values(ascending=False))
nation_list = {}
nation_index = 0
for k, i in nation_cnt.items():
    nation_list[k] = nation_index
    nation_index += 1
print(nation_list)
train_data['Nation'] = train_data['NOC'].apply(lambda x: nation_list[x])

train_data_path = './dataset/train_data.csv'
cols = ['Name', 'Age', 'Nation', 'Weight', 'Height', 'BMI', 'Year', 'Round', 'Wind', 'isHometown', 'Label']
train_data = train_data[cols]
train_data.to_csv(train_data_path, index=False)
train_data


{'USA': 0, 'GBR': 1, 'JAM': 2, 'FRA': 3, 'CAN': 4, 'NGR': 5, 'TTO': 6, 'BRA': 7, 'URS': 8, 'JPN': 9, 'GHA': 10, 'POL': 11, 'CUB': 12, 'AUS': 13, 'BAH': 14, 'GDR': 15, 'ITA': 16, 'GER': 17, 'CIV': 18, 'CHN': 19, 'FRG': 20, 'SKN': 21, 'BAR': 22, 'HUN': 23, 'SEN': 24, 'INA': 25, 'VEN': 26, 'ESP': 27, 'RSA': 28, 'KEN': 29, 'QAT': 30, 'GRE': 31, 'ANT': 32, 'CMR': 33, 'POR': 34, 'BEL': 35, 'NAM': 36, 'PAK': 37, 'BUL': 38, 'DOM': 39, 'PAN': 40, 'SUR': 41, 'GAM': 42, 'CGO': 43, 'UGA': 44, 'CYP': 45, 'TPE': 46, 'THA': 47, 'ISV': 48, 'SGP': 49, 'AHO': 50, 'IRI': 51, 'PUR': 52, 'MAD': 53, 'SUI': 54, 'MDV': 55, 'UKR': 56, 'NZL': 57, 'CHI': 58, 'MAS': 59, 'BER': 60, 'NOR': 61, 'ZAM': 62, 'BUR': 63, 'LBR': 64, 'SLE': 65, 'KSA': 66, 'NED': 67, 'KOR': 68, 'OMA': 69, 'TCH': 70, 'GAB': 71, 'MLI': 72, 'BAN': 73, 'TGA': 74, 'RUS': 75, 'GUY': 76, 'EUN': 77, 'FIJ': 78, 'CAY': 79, 'MEX': 80, 'ISL': 81, 'GUI': 82, 'BEN': 83, 'MRI': 84, 'ARG': 85, 'AUT': 86, 'SWE': 87, 'TUR': 88, 'LES': 89, 'PHI': 90, 'BOL': 9

,Name,Age,Nation,Weight,Height,BMI,Year,Round,Wind,isHometown,Label
0,Harrison Dillard,25.062286,0,69.0,178.0,21.777553,1948,1,0.9,0.0,10.40
1,Harrison Dillard,25.062286,0,69.0,178.0,21.777553,1948,2,0.0,0.0,10.40
2,Harrison Dillard,25.062286,0,69.0,178.0,21.777553,1948,3,0.9,0.0,10.50
3,Harrison Dillard,25.062286,0,69.0,178.0,21.777553,1948,4,1.6,0.0,10.50
4,Barney Ewell,30.425736,0,71.0,180.0,21.913580,1948,1,1.1,0.0,10.50
...,...,...,...,...,...,...,...,...,...,...,...
2291,Didier Kiki,25.667351,83,84.0,185.0,24.543462,2020,1,0.0,0.0,10.69
2292,Karalo Maibuca,22.140999,156,64.0,176.0,20.661157,2020,1,0.9,0.0,11.42
2293,Chijindu Ujah,27.405886,1,81.0,182.0,24.453568,2020,2,0.8,0.0,10.08
2294,Chijindu Ujah,27.405886,1,81.0,182.0,24.453568,2020,3,0.8,0.0,10.11
